# Highway Volumes Reports

- Highway volumes (single report or comparison between 2 scenarios)
    - standard
        - aggregate (MPO, sub-region, land use classification) 
        - segmented by facility type 
    - Detailed
        - standard report for user-select links (allow for more than 1 selection set (1,2,3,4..))

- Data
    - highway assignment results
        - SOV, HOV assignment: Out\**_MMA_LinkFlow.bin
        - Truck assignment: Out\**_MMA_LinkFlow_Trucks.bin
            - sum the truck and SOV and HOV
            - (look at Tot_Flow, AB_Flow, BA_Flow)


** = AM, MD, PM, NT

??? = DAT, DET


In [ ]:
import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
from io import StringIO
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import cartopy.crs as ccrs
import csv
import plotly.express as px
import plotly.graph_objects as go

Please edit config.py before using this notebook!

Note: set comparison_scenario_dir to '' in the config file if not using.

In [ ]:
# ===>>>USER INPUT REQUIRED: <<<===

#update to your file path to confix.py
%run "M:/JupyterHome/JupyterNotebooks/MoDX/config.py"

## Name of file containing detailed links (if using) in base_scenario_dir and comparison_scenario_dir (if have comp scenario)
#IF NOT USING - SET BOTH TO ''
hwy_links_file = ''#reference_data_dir + 'sample_model_links/highway info.csv'
ID_FIELD = ''#'TC Link ID'

# END USER INPUT #

In [ ]:
# Base directory for MoDX output for "base year" model results. (From config.py)
#
print(base_scenario_dir) #Original: r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
#
# Base directory for MoDX output for "comparison scenario" model results. (Optional) (From config.py)
# 
print(comparison_scenario_dir) #Original: r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'

#bring in TAZs for later use (and so don't have to repeat)
taz = reference_data_dir + "canonical_taz_shapefile/candidate_CTPS_TAZ_STATEWIDE_2019.shp"

In [ ]:
#Set up all the directories that are needed and start bringing in data
def dataSetup(base):
    #IMPORT links and taz spatial and attribute DATA
    links = base + "Databases/Statewide_Links_2018.shp" #this will actually be in the scen directory

    #load into a geodataframe
    links_gdf = gp.read_file(links)
    taz_gdf = gp.read_file(taz)
    taz_gdf=taz_gdf.loc[taz_gdf['type']=='I'] #restrict just to internal tazs

    #make sure crs are correct and match
    links_gdf=links_gdf.to_crs('EPSG:26986')
    taz_gdf=taz_gdf.to_crs('EPSG:26986')

    #do attribute join to find out TAZ details for the taz each link is in
    links_taz_gdf = links_gdf.merge(taz_gdf.loc[:, ~taz_gdf.columns.isin(['geometry', 'OBJECTID', 'Shape_Leng', 'Shape_Area'])], 
                                    left_on='TAZ_ID', right_on='taz') #exclude all columns listed above
    #filter to links that are in internal tazs and that have taz information at all
    highway_links_df = links_taz_gdf.loc[(links_taz_gdf['taz'] >0) & (links_taz_gdf['type']=='I') & 
                                         (links_taz_gdf['in_brmpo'] ==1)]
    #rename ID to ID1 to make everything else work later in script
    highway_links_df=highway_links_df.rename(columns={'ID': 'ID1'}) #won't fail if the column doesn't exist
    #if detailed, filter
    if len(hwy_links_file) > 4:
        hwy_links_detail = pd.read_csv(hwy_links_file) #read in detail file
        highway_links_list = hwy_links_detail[ID_FIELD].tolist() #grab just the ID field
        
        highway_links_df = highway_links_df.loc[highway_links_df['ID1'].isin(highway_links_list)]
    return highway_links_df, links_taz_gdf

In [ ]:
def hwyVolSpeVOC(scen, hwy_links_file):
        # Individual link-flow CSV tables:
    # For each time period, there is a separate flow CSV for autos and for trucks.
    # To get the total volume for any given time period, 'Tot_Flow' columns these need to be summed.
    # However, the V/C and speed data for *both* autos and trucks are reported in the CSV for autos.
    # Clear?
    
    home_dir = scen #whichever scenario

    # Directory containing link flow CSVs
    link_flow_dir = home_dir + 'out/'
    #
    am_flow_auto_fn = link_flow_dir + 'AM_MMA_LinkFlow.csv'
    am_flow_truck_fn = link_flow_dir + 'AM_MMA_LinkFlow_Trucks.csv'
    #
    md_flow_auto_fn = link_flow_dir + 'MD_MMA_LinkFlow.csv'
    md_flow_truck_fn = link_flow_dir + 'MD_MMA_LinkFlow_Trucks.csv'
    #
    pm_flow_auto_fn = link_flow_dir + 'PM_MMA_LinkFlow.csv'
    pm_flow_truck_fn = link_flow_dir + 'PM_MMA_LinkFlow_Trucks.csv'
    #
    nt_flow_auto_fn = link_flow_dir + 'NT_MMA_LinkFlow.csv'
    nt_flow_truck_fn = link_flow_dir + 'NT_MMA_LinkFlow_Trucks.csv'

    # Read each of the above CSV files containing flow data into a dataframe
    #
    am_auto_df = pd.read_csv(am_flow_auto_fn, delimiter=',')
    am_truck_df = pd.read_csv(am_flow_truck_fn, delimiter=',')
    #
    md_auto_df = pd.read_csv(md_flow_auto_fn, delimiter=',')
    md_truck_df = pd.read_csv(md_flow_truck_fn, delimiter=',')
    #
    pm_auto_df = pd.read_csv(pm_flow_auto_fn, delimiter=',')
    pm_truck_df = pd.read_csv(pm_flow_truck_fn, delimiter=',')
    #
    nt_auto_df = pd.read_csv(nt_flow_auto_fn, delimiter=',')
    nt_truck_df = pd.read_csv(nt_flow_truck_fn, delimiter=',') 
    
        #
        # NOTE: volume/capacity and speed data will be harvested from the "auto" dataframes subsequently.
        #       See below.

    # Filter the "flow" datafames to only include the columns containing 'Tot_Flow' and 'Tot_VMT'(and 'ID1')
    # 
    am_auto_vol_df = am_auto_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    am_truck_vol_df = am_truck_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    #
    md_auto_vol_df = md_auto_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    md_truck_vol_df = md_truck_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    #
    pm_auto_vol_df = pm_auto_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    pm_truck_vol_df = pm_truck_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    #
    nt_auto_vol_df = nt_auto_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    nt_truck_vol_df = nt_truck_df[['ID1', 'Tot_Flow', 'Tot_VMT']]
    
    # Rename the 'Tot_Flow' column of each dataframe, appropriately
    #
    am_auto_vol_df = am_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_am_auto', 'Tot_VMT' : 'Tot_VMT_am_auto'})
    am_truck_vol_df = am_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_am_truck', 'Tot_VMT' : 'Tot_VMT_am_truck'})
    #
    md_auto_vol_df = md_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_md_auto', 'Tot_VMT' : 'Tot_VMT_md_auto'})
    md_truck_vol_df = md_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_md_truck', 'Tot_VMT' : 'Tot_VMT_md_truck'})
    #
    pm_auto_vol_df = pm_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_pm_auto', 'Tot_VMT' : 'Tot_VMT_pm_auto'})
    pm_truck_vol_df = pm_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_pm_truck', 'Tot_VMT' : 'Tot_VMT_pm_truck'})
    #
    nt_auto_vol_df = nt_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_nt_auto', 'Tot_VMT' : 'Tot_VMT_nt_auto'})
    nt_truck_vol_df = nt_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_nt_truck', 'Tot_VMT' : 'Tot_VMT_nt_truck'})


    # Index all the "volume" dataframes on "ID1", in preparation for joining
    #
    am_auto_vol_df.set_index("ID1")
    am_truck_vol_df.set_index("ID1")
    #
    md_auto_vol_df.set_index("ID1")
    md_truck_vol_df.set_index("ID1")
    #
    pm_auto_vol_df.set_index("ID1")
    pm_truck_vol_df.set_index("ID1")
    #
    nt_auto_vol_df.set_index("ID1")
    nt_truck_vol_df.set_index("ID1")

    # Join the "volume" dataframes
    j1_df = am_auto_vol_df.join(am_truck_vol_df.set_index("ID1"), on="ID1")
    #
    j1_df.set_index("ID1")
    j2_df = j1_df.join(md_auto_vol_df.set_index("ID1"), on="ID1")
    #
    j2_df.set_index("ID1")
    j3_df = j2_df.join(md_truck_vol_df.set_index("ID1"), on="ID1")
    #
    j3_df.set_index("ID1")
    j4_df = j3_df.join(pm_auto_vol_df.set_index("ID1"), on="ID1")
    #
    j4_df.set_index("ID1")
    j5_df = j4_df.join(pm_truck_vol_df.set_index("ID1"), on="ID1")
    #
    j5_df.set_index("ID1")
    j6_df = j5_df.join(nt_auto_vol_df.set_index("ID1"), on="ID1")
    #
    j6_df.set_index("ID1")
    total_flow_join = j6_df.join(nt_truck_vol_df.set_index("ID1"), on="ID1")
    
    # Calculate the total volume (auto + truck) for each time period, and for the entire day
    #
    total_flow_join['Tot_Flow_am'] = total_flow_join['Tot_Flow_am_auto'] + total_flow_join['Tot_Flow_am_truck']
    #
    total_flow_join['Tot_Flow_md'] = total_flow_join['Tot_Flow_md_auto'] + total_flow_join['Tot_Flow_md_truck']
    #
    total_flow_join['Tot_Flow_pm'] = total_flow_join['Tot_Flow_pm_auto'] + total_flow_join['Tot_Flow_pm_truck']
    #
    total_flow_join['Tot_Flow_nt'] = total_flow_join['Tot_Flow_nt_auto'] + total_flow_join['Tot_Flow_nt_truck']
    #
    total_flow_join['Tot_Flow_daily'] = (total_flow_join['Tot_Flow_am'] + total_flow_join['Tot_Flow_md'] + 
                                         total_flow_join['Tot_Flow_pm'] + total_flow_join['Tot_Flow_nt'])
    
    # Calculate the total volume (auto + truck) for each time period, and for the entire day (VMT STYLE!)
    #
    total_flow_join['Tot_VMT_am'] = total_flow_join['Tot_VMT_am_auto'] + total_flow_join['Tot_VMT_am_truck']
    #
    total_flow_join['Tot_VMT_md'] = total_flow_join['Tot_VMT_md_auto'] + total_flow_join['Tot_VMT_md_truck']
    #
    total_flow_join['Tot_VMT_pm'] = total_flow_join['Tot_VMT_pm_auto'] + total_flow_join['Tot_VMT_pm_truck']
    #
    total_flow_join['Tot_VMT_nt'] = total_flow_join['Tot_VMT_nt_auto'] + total_flow_join['Tot_VMT_nt_truck']
    #
    total_flow_join['Tot_VMT_daily'] = (total_flow_join['Tot_VMT_am'] + total_flow_join['Tot_VMT_md'] + 
                                         total_flow_join['Tot_VMT_pm'] + total_flow_join['Tot_VMT_nt'])

    #set index
    total_flow_join.set_index("ID1")


    # Harvest the speed and volume-to-capacity ratio data from the 4 "auto" dataframes, one for each time period. (See above.)
    # Note Python variable naming convention used here: "svc" == "speed and volume/capacity"
    #
    am_svc_df = am_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
    #
    md_svc_df = md_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
    #
    pm_svc_df = pm_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
    #
    nt_svc_df = nt_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]

    # Rename the columns of these "svc" dataframes in preparation for joining them with the speed dataframe, computed above.
    #
    am_svc_df = am_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_am', 
                                          'BA_Speed' : 'BA_Speed_am',
                                          'AB_VOC'   : 'AB_VOC_am', 
                                          'BA_VOC'   : 'BA_VOC_am'})
    #
    md_svc_df = md_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_md', 
                                          'BA_Speed' : 'BA_Speed_md',
                                          'AB_VOC'   : 'AB_VOC_md', 
                                          'BA_VOC'   : 'BA_VOC_md'})
    #
    pm_svc_df = pm_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_pm', 
                                          'BA_Speed' : 'BA_Speed_pm',
                                          'AB_VOC'   : 'AB_VOC_pm', 
                                          'BA_VOC'   : 'BA_VOC_pm'})
    #
    nt_svc_df = nt_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_nt', 
                                          'BA_Speed' : 'BA_Speed_nt',
                                          'AB_VOC'   : 'AB_VOC_nt', 
                                          'BA_VOC'   : 'BA_VOC_nt'})
    
    # Per instructions from Marty on June 22, 2021:
    # For a given time period, calculate the MIN of the AB_Speed and BA_Speed, and the MAX of the AB_VOC and BA_VOC.
    # Basically, the idea is to flag the link direction with the most congestion.
    #
    am_svc_df['Speed_am'] = am_svc_df.apply(lambda x: min(x['AB_Speed_am'], x['BA_Speed_am']), axis=1)
    am_svc_df['VOC_am'] = am_svc_df.apply(lambda x: max(x['AB_VOC_am'], x['BA_VOC_am']), axis=1)
    #
    md_svc_df['Speed_md'] = md_svc_df.apply(lambda x: min(x['AB_Speed_md'], x['BA_Speed_md']), axis=1)
    md_svc_df['VOC_md'] = md_svc_df.apply(lambda x: max(x['AB_VOC_md'], x['BA_VOC_md']), axis=1)
    #
    pm_svc_df['Speed_pm'] = pm_svc_df.apply(lambda x: min(x['AB_Speed_pm'], x['BA_Speed_pm']), axis=1)
    pm_svc_df['VOC_pm'] = pm_svc_df.apply(lambda x: max(x['AB_VOC_pm'], x['BA_VOC_pm']), axis=1)
    #
    nt_svc_df['Speed_nt'] = nt_svc_df.apply(lambda x: min(x['AB_Speed_nt'], x['BA_Speed_nt']), axis=1)
    nt_svc_df['VOC_nt'] = nt_svc_df.apply(lambda x: max(x['AB_VOC_nt'], x['BA_VOC_nt']), axis=1)

    # Index the "svc" dataframes in preparation for joining
    am_svc_df.set_index("ID1")
    md_svc_df.set_index("ID1")
    pm_svc_df.set_index("ID1")
    nt_svc_df.set_index("ID1")

    # Join the speed and volume/capacity data to the volume data collected above into a single dataframe.
    #
    j7_df = total_flow_join.join(am_svc_df.set_index("ID1"), on="ID1")
    j7_df.set_index("ID1")
    #
    j8_df = j7_df.join(md_svc_df.set_index("ID1"), on="ID1")
    j8_df.set_index("ID1")
    #
    j9_df = j8_df.join(pm_svc_df.set_index("ID1"), on="ID1")
    j9_df.set_index("ID1")
    #
    all_data_df = j9_df.join(nt_svc_df.set_index("ID1"), on="ID1")
    
    return all_data_df

In [ ]:
def hwyGraphs(all_data_df, scen):
#restrict to just volume (vmt) data for standard graphs and aggregation numbers
    links4graph=highway_links_df.merge(all_data_df[['ID1','Tot_VMT_am', 'Tot_VMT_md',
                                            'Tot_VMT_pm','Tot_VMT_nt','Tot_VMT_daily']],
                               how = 'left', left_on ='ID1', right_on='ID1')

    if len(hwy_links_file) < 4: #STANDARD
            #make graphs
        #first aggregate to functional class
        volFU = links4graph.groupby(['SCEN_00_FU'])[['Tot_VMT_am', 'Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt','Tot_VMT_daily']].agg('sum')
        volFU = volFU.reset_index()

        #make func class into categories
        volFU['Functional Class'] = np.where(volFU['SCEN_00_FU'].isin([72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]), 'Ramps',
                                            np.where(volFU['SCEN_00_FU'].isin([50,51,52,53,54,55,56]), 'HOV', 
                                                    np.where(volFU['SCEN_00_FU'].isin([1,2,3,5,6,10]), volFU['SCEN_00_FU'], 'Other')))
        volFU['Functional Class'] = volFU['Functional Class'].apply(str)
        volFU = volFU.groupby(['Functional Class'])[['Tot_VMT_am', 'Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt','Tot_VMT_daily']].agg('sum')
        volFU = volFU.reset_index()

        volFUgraph = px.bar(volFU, 
                        x = 'Functional Class', y = ['Tot_VMT_am','Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt'], 
                        title=scen +' Volumes by Functional Class/Facility Type', labels = {'value':'VMT'},
                            hover_data=['Tot_VMT_daily'])
    else: #DETAILED
        volFU =links4graph #no group by because individual links
        volFU['ID1'] = volFU['ID1'].astype('str')
        volFUgraph = px.bar(volFU, 
                        x = ['Tot_VMT_am','Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt'], y = 'ID1', 
                        title=scen +' Volumes for Selected Link IDs', height = 1000, 
                        labels = {'ID1': 'Link ID', 'value':'VMT'}, orientation ='h', hover_data=['Tot_VMT_daily'])
        volFUgraph.update_yaxes(type='category')
        
    return volFUgraph, links4graph


In [ ]:
def makeNums(all_data_df, links4graph):
    #get aggregation numbers
    if len(hwy_links_file) < 4:
        #filter for aggregation numbers
        links4num = links4graph.loc[links4graph['in_brmpo']==1]
        #volume sums for the MPO
        volMPO=links4num.groupby(['in_brmpo'])[['Tot_VMT_am', 'Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt','Tot_VMT_daily']].agg('sum')

        #volume sums for the MPO subregions
        volSubReg=links4num.groupby(['subregion'])[['Tot_VMT_am', 'Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt','Tot_VMT_daily']].agg('sum')

        #volume sums for all the TAZs
        volTAZ=links4num.groupby(['taz'])[['Tot_VMT_am', 'Tot_VMT_md','Tot_VMT_pm','Tot_VMT_nt','Tot_VMT_daily']].agg('sum')
    else:
        volMPO = 'Please run STANDARD version to get VMT at these geographies'
        volSubReg = 'Please run STANDARD version to get VMT at these geographies'
        volTAZ = 'Please run STANDARD version to get VMT at these geographies'
    
    return volMPO, volSubReg, volTAZ

In [ ]:
#calculate difference graphs and tables
def calcSum(outputs):
    #get difference between scenarios at a link or functional class level
    dif = outputs['Base Scenario'][0]-outputs['Comparison Scenario'][0]
    if len(hwy_links_file) < 4:
        #get the difference between the key summary outputs only if STANDARD
        #do not include geometry field - empty and has issues for difference.
        volMPO1 = pd.DataFrame(outputs['Base Scenario'][2])-pd.DataFrame(outputs['Comparison Scenario'][2])
        volSubReg1 = pd.DataFrame(outputs['Base Scenario'][3])-pd.DataFrame(outputs['Comparison Scenario'][3])
        volTAZ1 = pd.DataFrame(outputs['Base Scenario'][4])-pd.DataFrame(outputs['Comparison Scenario'][4])
        
        #create summary tables (already exist if just base)
        #two steps to each:
        #1. SubRegions of MPO
        volSubReg = pd.DataFrame(outputs['Base Scenario'][3]).merge(pd.DataFrame(outputs['Comparison Scenario'][3]), how = 'outer', on='subregion', suffixes = ('_base', '_comp'))
        volSubReg1 =volSubReg1.add_suffix('_dif') #differentiate columns for diff
        volSubReg = volSubReg.merge(volSubReg1, how = 'outer',on='subregion')
        #2 TAZs of MPO
        volTAZ= pd.DataFrame(outputs['Base Scenario'][4]).merge(pd.DataFrame(outputs['Comparison Scenario'][4]), how = 'outer',on='taz', suffixes = ('_base', '_comp'))
        volTAZ1 =volTAZ1.add_suffix('_dif') #differentiate columns for diff
        volTAZ= volTAZ.merge(pd.DataFrame(volTAZ1), how = 'outer',on='taz')
        ##3. MPO Aggregation (just one row so append)
            #first update index
        outputs['Base Scenario'][2]['Scenario']='Base'
        outputs['Comparison Scenario'][2]['Scenario']='Comparison'
        volMPO1['Scenario']='Difference'
            #now actually do the table
        volMPO = pd.DataFrame(outputs['Base Scenario'][2]).append(pd.DataFrame(outputs['Comparison Scenario'][2]))
        volMPO = volMPO.append(pd.DataFrame(volMPO1))
        
    else: #IF DETAILED (aka links selected)
        volMPO = 'Please run STANDARD version to get VMT at these geographies'
        volSubReg = 'Please run STANDARD version to get VMT at these geographies'
        volTAZ = 'Please run STANDARD version to get VMT at these geographies'
            
    #create a difference graph
    graph, links4graph = hwyGraphs(dif, 'Difference')
    #save results into dictionary
    outputs['Difference'] = [dif, graph, volMPO, volSubReg, volTAZ]
    
    return outputs, volMPO, volSubReg, volTAZ

## Run All Functions (Create the Report Data)

In [ ]:
#Only use second scenario if provided
if len(comparison_scenario_dir) > 0:
    scenlist =[base_scenario_dir, comparison_scenario_dir]
else:
    scenlist = [base_scenario_dir]

#create output dictionary
outputs = {}
for scen in scenlist:
    #import and clean links and tazs data tables for the scenario
    highway_links_df, links_taz_gdf = dataSetup(scen)
    #join a bunch of speed, volume, flow, and calculate VOC
    all_data_df = hwyVolSpeVOC(scen, hwy_links_file)
    #title of scenario is important for graphing in the title
    if scen == scenlist[0]:
        s = 'Base Scenario'
    else:
        s = 'Comparison Scenario'
    graph, links4graph = hwyGraphs(all_data_df, s) #make the graph!
    volMPO, volSubReg, volTAZ = makeNums(all_data_df, links4graph) #make the summary numbers!
    outputs[s] = [all_data_df, graph, volMPO, volSubReg, volTAZ] #save it all!
#only do difference if two scenarios:
if len(scenlist) > 1:
    #calculate the summary tables and add difference to everything
    outputs, volMPO, volSubReg, volTAZ = calcSum(outputs)




## See Resulting Graph(s)

In [ ]:
outputs['Base Scenario'][1].show()

In [ ]:
#OPTIONAL - ONLY RUN IF COMPARISON SCENARIO
try:
    outputs['Comparison Scenario'][1].show()
except KeyError:
    print('No Comparison Scenario Input')

In [ ]:
#OPTIONAL - ONLY RUN IF COMPARISON SCENARIO
try:
    outputs['Difference'][1].show()
except KeyError:
    print('No Comparison Scenario Input')

## Show Summary Numbers:

In [ ]:
#MPO
volMPO

In [ ]:
#SubRegions
volSubReg

In [ ]:
#TAZ
volTAZ